<a href="https://colab.research.google.com/github/saatweek/Intel-Image-Classification/blob/main/Intel_Image_Classification_(Using_Convolutions).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#The dataset is stored in my Google Drive, so we'll first have to connect this notebook 
#to the Google drive to access the files in the drive from this notebook 

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#importing dependencies

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import plotly.graph_objects as go

In [ ]:
#Implementing a Callback to stop training when the desired training accuracy is reached

class myCallback (tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = {}):
    if (logs.get('acc')>0.95):
      self.model.stop_training = True
      print('Enough Accuracy Reached!')

callback = myCallback()

In [ ]:
#Image Data Generator

train_dir = '/content/drive/My Drive/Colab Notebooks/Personal Projects/Intel Image Classification/seg_train'
validation_dir = '/content/drive/My Drive/Colab Notebooks/Personal Projects/Intel Image Classification/seg_test'

train_datagen = ImageDataGenerator(rescale = 1./255, 
                                   rotation_range=0.3, 
                                   width_shift_range=0.3,
                                   height_shift_range=0.3,
                                   shear_range=0.3, 
                                   zoom_range=0.3, 
                                   horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale = 1./255, 
                                        rotation_range=0.3, 
                                        width_shift_range=0.3,
                                        height_shift_range=0.3,
                                        shear_range=0.3, 
                                        zoom_range=0.3, 
                                        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size = (150, 150), 
                                                    batch_size = 128, 
                                                    class_mode='sparse')

validation_generator = validation_datagen.flow_from_directory(validation_dir, 
                                                              target_size = (150, 150), 
                                                              batch_size = 128, 
                                                              class_mode='sparse')

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(16, (3, 3), activation = 'relu', input_shape = (150, 150, 3)),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(256, activation='relu'),
                                    tf.keras.layers.Dense(6, activation = 'softmax')])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy', 
              optimizer='adam', 
              metrics = 'acc')

In [ ]:
history = model.fit(train_generator, 
                    steps_per_epoch=20, 
                    epochs=40, 
                    validation_data = validation_generator, 
                    callbacks = [callback])

Epoch 1/40
20/20 [==============================] - 1335s 67s/step - loss: 1.4986 - acc: 0.3684 - val_loss: 1.2801 - val_acc: 0.4927
Epoch 2/40
20/20 [==============================] - 534s 27s/step - loss: 1.2317 - acc: 0.5098 - val_loss: 1.1513 - val_acc: 0.5437
Epoch 3/40
20/20 [==============================] - 422s 21s/step - loss: 1.1206 - acc: 0.5551 - val_loss: 1.0810 - val_acc: 0.5827
Epoch 4/40
20/20 [==============================] - 373s 19s/step - loss: 1.0316 - acc: 0.5934 - val_loss: 1.0318 - val_acc: 0.5953
Epoch 5/40
20/20 [==============================] - 323s 16s/step - loss: 0.9887 - acc: 0.6062 - val_loss: 0.9791 - val_acc: 0.6160
Epoch 6/40
20/20 [==============================] - 262s 13s/step - loss: 0.9376 - acc: 0.6395 - val_loss: 1.0035 - val_acc: 0.6190
Epoch 7/40
20/20 [==============================] - 214s 11s/step - loss: 0.9533 - acc: 0.6500 - val_loss: 0.9553 - val_acc: 0.6430
Epoch 8/40
20/20 [==============================] - 186s 9s/step - loss: 0.

In [ ]:
#PLotting the graph

fig = go.Figure()
x = np.linspace(1,20, 20)
y1 = history.history['acc']
y2 = history.history['val_acc']
fig.add_trace(go.Scatter(x = x, y = y1, name = 'training accuracy')),
fig.add_trace(go.Scatter(x = x, y = y2, name = 'validation accuracy')),
fig.update_layout(xaxis_title = 'Epochs', yaxis_title = 'accuracy', title = 'Accuracy of Model')
fig.show()